In [1]:
# Setting up a custom stylesheet in IJulia
# New in 0.6
file = open("style.css") # A .css file in the same folder as this notebook file
styl = readstring(file) # Read the file
HTML("$styl") # Output as HTML

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n#notebook_panel { /* main background */\r\n    background: #ddd;\r\n    color: #000000;\r\n}\r\n\r\n\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.CodeMirror{\r\n        font-family: \"PT Mono\";\r\n        font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

<h1> Plotting data and an approximately fitted line simultaneously </h1>

<h2>In this lecture</h2>

- [Outcome](#Outcome)
- [Julia's modify-the-argument functions](#Julia's-modify-the-argument-functions)
- [The difference between plot() and plot!()](#The-difference-between-"plot"-and-"plot!")
- [An example: approximating the cosine function](#An-example:-approximating-the-cosine-function)

[Back to the top](#In-this-lecture)

<h2>Outcome</h2>

After this lecture, you will be able to
- Explain how Julia's plot!() function differs from plot()
- Use plot!() to overlay a fitted curve to a scatter of data
- Modify the plot: adding labels, axis styles, line styles etc


[Back to the top](#In-this-lecture)

<h2>Julia's modify-the-argument functions</h2>

A lovely feature of Julia is the convention of labelling modify-the-argument functions with an exclamation mark.

For example, consider the function fill!(), illustrated below.

In [28]:
# New in 0.6
##tempvar = Array{Any}(4)
tempvar = zeros(Int64, 1, 4) #Array{Int64, 4}(0, 4, 1)
Array1 = [1, 2, 3, 4, 5]
Array2 = [1 2; 3 4]
##fill!(tempvar, "hello ... word")

println(typeof(tempvar))
println(typeof(Array1))
println(typeof(Array2))

tempvar = fill!(tempvar, 7)
println(tempvar)
Array1 = fill!(Array1, 10)
println(Array1)
Array2 = fill!(Array2, 10)
println(Array2)

Matrix{Int64}
Vector{Int64}
Matrix{Int64}
[7 7 7 7]
[10, 10, 10, 10, 10]
[10 10; 10 10]


In [29]:
fill!(tempvar, 777)

1×4 Matrix{Int64}:
 777  777  777  777

Let's just look at how the help system briefly, comparing these two functions and also looking at all functions starting with "a" and with "b" to see how many of the modify-in-place functions Julia has.

In [5]:
?b

search: bin big bits bind bfft beta base break begin Bool bswap brfft bfft! Base

Couldn't find b
Perhaps you meant !, $, %, &, *, +, -, /, :, <, >, I, \, ^, e, |, ~, ×, ÷ or γ


No documentation found.

Binding `b` does not exist.


We see that quite a few functions have a plain form and an exclamation form. And many of the exclamation-ending-modify-in-place functions do things like add or remove values. They tend to be low level functions. The advantage of course is that as values are created they are written to space in memory that is already in use. This can speed up one's code significantly, and can also be important when your data sets are so large that memory efficiency becomes an issue.

[Back to the top](#In-this-lecture)

<h2>The difference between "plot" and "plot!"</h2>

As we remarked, Plots is a package for using all the Julia plotting packages. The designer of Plots made the interesting choice (I think) to have only one active plot: if you want to make a different plot, you save your current plot and start on a new one. This simplifies the display of a plot. In our case, the notebook allows us to view different plots, because each plot is saved with its particular cell. In keeping with the idea of Plots, you cannot associate more than one plot with a cell.

So in general, if you call plot() a second time, it wipes out the old plot and draws a completely new one. But Plots *also* doesn't allow you to specify multiple different x-values for your plot. You can plot many different curves, as we did for the different countries' Ebola numbers (week 2, lecture 5), but only if they all use the same x-values.

To enable us to plot completely different curves, not only in style and colour and so on but also completely different x- and y-values, Plots gives us a function that modifies an existing plot. This function is plot!().

So the idea is the following: specify a plot for the first set of data using "plot()", and then specify, on the same plot, a second set of data by using "plot!"

[Back to the top](#In-this-lecture)

<h2>An example: approximating the cosine function</h2>

An example will make this clear. It is well known that the cosine function has approximations based on  Taylor series. We will use the fact that 

$cos(x) \approx 1 - \dfrac{x^2}{2} + \dfrac{x^4}{24} - \dfrac{x^6}{720}$

is an excellent approximation as long as $x$ is near zero. We will pretend the approximate values are experimental data, so there are just a few points. Then we compare the actual function as a curve, and compare them.

While we're at it, let's practice writing functions. We will write a function that takes a whole vector as input, and one by one evaluates the elements using the approximation formula, and then returns an output vector of the same size as the input.

In [43]:
# New in 0.6
function approxcos(x)
    #initialise the output ... note the use of size() to specify the dimensions of the output vector
    outval = zeros(size(x))#Array{Any, 1}(size(x), 1)  
    
    # now we loop over the input vector, and for each  element calculate and store the approximation
    ii = 0  # this will be the index into the vector
    for aa in x   # this aa is just a number, an element of the vector
        y = 1 - aa^2/2 + aa^4/24 - aa^6/720 + aa^8/(56*720) # the approximation ...
        ii = ii+1            #this sets the index correctly
        outval[ii] = y     # and this stores the approximation in the right place
    end
    return outval  
end

x1 = 6rand(10)  # rand() is one of several random number functions in Julia. It returns numbers that uniformly fill the interval [0, 1]
#                   .... here we use it get a set of sampling points in the interval [0, 4]
println(x1) 
x2 = range(0,stop=6,length=20) #linspace(0, 6, 200)   # look up linspace() using "?" ... it's a nice way to get evenly spaced points


y1 = approxcos(x1)
println(y1)
#y2 = cos.(x2)


[2.0474677331814086, 2.302604769813705, 4.6962926407818735, 3.7984413049098205, 1.2814261472210684, 2.5874609864329625, 2.77066714607822, 3.7984756468143206, 1.5888412537985341, 2.4457182044776364]
[-0.4584784658472347, -0.6671066053707528, 1.2083428269066623, -0.6370441398117492, 0.28535189613067097, -0.8468323568439475, -0.9250555528113473, -0.6370094574841938, -0.01801622906544745, -0.7654758853586972]


In [45]:
# now the plots
using Plots;     # it is sometimes convenient to cram a line in this way

#first the plot of the approximation points
plot(x1, y1)

#plot!(x2, y2, line = (:path, :red))   #then add the accurate line with plot!()

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/lali/.julia/packages/PyCall/BD546/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'Text' object has no property 'math_fontfamily'")
  File "/home/lali/anaconda3/lib/python3.8/site-packages/matplotlib/axes/_base.py", line 63, in wrapper
    return get_method(self)(*args, **kwargs)
  File "/home/lali/anaconda3/lib/python3.8/site-packages/matplotlib/cbook/deprecation.py", line 451, in wrapper
    return func(*args, **kwargs)
  File "/home/lali/anaconda3/lib/python3.8/site-packages/matplotlib/axis.py", line 1793, in _set_ticklabels
    return self.set_ticklabels(labels, minor=minor, **kwargs)
  File "/home/lali/anaconda3/lib/python3.8/site-packages/matplotlib/axis.py", line 1740, in set_ticklabels
    tick.label1.update(kwargs)
  File "/home/lali/anaconda3/lib/python3.8/site-packages/matplotlib/text.py", line 177, in update
    super().update(

[Back to the top](#In-this-lecture)